# **Black Wednesday**

### **1. Import and clean Data**

In [38]:
import pandas as pd
from lets_plot import *
LetsPlot.setup_html()

In [ ]:
UK_data = pd.read_excel("../data/UK.xlsx", skiprows=1).rename(columns={
    "Unnamed: 1": "Month",
    "Economic Activity, Industrial Production, Index": "Industrial_Production",
    "Exchange Rates, National Currency Per U.S. Dollar, Period Average, Rate": "National_Exchange_Rate",
    "Exchange Rates, Real Effective Exchange Rate based on Consumer Price Index, Index": "Real_Effective_Exchange_Rate",
    "Prices, Consumer Price Index, All items, Index": "CPI"
}).drop("Unnamed: 0", axis=1)

# The data includes average for the year, not just monthly data so we remove these rows.
UK_data = UK_data[~UK_data['Month'].str.match(r'^\d{4}$')]

# Convert the dates to datetime objects so it is easier to work with.
UK_data['Date'] = pd.to_datetime(UK_data['Month'], format='%b %Y')
UK_data = UK_data.drop("Month", axis=1)

UK_data['Real_Exchange_Rate_Growth'] = UK_data['Real_Effective_Exchange_Rate'].pct_change() * 100

### **2. Is there evidence that the UK joining the ERM and leaving after Black Wednesday led to changes in the volatility of the monthly growth in the real exchange rate?**

In [ ]:
date_joined_ERM = pd.to_datetime("Oct 1990")
date_left_ERM = pd.to_datetime("Sept 1992")

In [92]:
level_plot = (
    ggplot(UK_data, aes(x='Date', y='Real_Exchange_Rate_Growth')) +
    ggtitle("Volatility decreased when they joined") +
    geom_line(color='#1f77b4', size=1.0) +  
    geom_vline(xintercept=date_joined_ERM, color='red', size=0.5) +
    geom_vline(xintercept=date_left_ERM,   color='red', size=0.5) +
    xlab('Date') +
    ylab('Real Effective Exchange Rate growth') +
    theme(
        panel_background=element_rect(fill='white', color='white'),
        panel_grid_major=element_line(color='lightgrey', size=0.5, linetype='dashed'),
        panel_grid_minor=element_line(color='white', size=0.25, linetype='dashed'),
        axis_text=element_text(size=10, color='black'),
        axis_title=element_text(size=12, color='black'),
        plot_title=element_text(size=16, face='bold', hjust=0.5),
        legend_background=element_rect(fill='none')
    ) +
    ggsize(800,500)
)

UK_data['Volatility'] = UK_data['Real_Effective_Exchange_Rate'].rolling(window=12).std()


volatility_plot = (
    ggplot(UK_data, aes(x='Date', y='Volatility')) +
    ggtitle("and increased (briefly) when they left") +
    geom_line(color='#1f77b4', size=1.0) +
    geom_vline(xintercept=date_joined_ERM, color='red', size=0.5) +
    geom_vline(xintercept=date_left_ERM, color='red', size=0.5) +
    xlab('Date') +
    ylab('Volatility (12-month Rolling STD)') +
    theme(
        panel_background=element_rect(fill='white', color='white'),
        panel_grid_major=element_line(color='lightgrey', size=0.5, linetype='dashed'),
        panel_grid_minor=element_line(color='white', size=0.25, linetype='dashed'),
        axis_text=element_text(size=10, color='black'),
        axis_title=element_text(size=12, color='black'),
        plot_title=element_text(size=16, face='bold', hjust=0.5),
        legend_background=element_rect(fill='none')
    ) +
    ggsize(800,500)
)

gggrid([level_plot, volatility_plot], ncol=2, widths=[800, 800], fit=True)

### **3. Are there any corresponding changes in the volatility of either the difference in inflation between the UK and Germany or the growth in industrial production?**

In [ ]:
German_UK_inflation = pd.DataFrame()
German_UK_inflation["Date"] = UK_data["Date"]
German_UK_inflation["UK_CPI_Inflation"] = (UK_data["CPI"] - UK_data["CPI"])/UK_data["CPI"] #This code is wrong, it needs to be the change in CPI from the year before, in the way CPI inflation is calculated.
German_UK_inflation["German_CPI_Inflation"] = 

KeyError: ('Date', 'CPI')